In [6]:
import tensorflow_datasets as tfds

# Carregar dataset de tradução (Português → Inglês)
examples, metadata = tfds.load("ted_hrlr_translate/pt_to_en", as_supervised=True, with_info=True)
train_data, val_data = examples["train"], examples["validation"]

In [7]:
import tensorflow as tf

# Criar dataset de frases traduzidas
frases_pt = ["Olá, como vai?", "Esse é um teste.", "Estamos aprendendo Transformers."]
frases_en = ["Hello, how are you?", "This is a test.", "We are learning Transformers."]

# Converter para TensorFlow Dataset
dataset = tf.data.Dataset.from_tensor_slices((frases_pt, frases_en))

# Exibir exemplos do dataset
for pt, en in dataset.take(3):
    print(pt.numpy(), "->", en.numpy())

b'Ol\xc3\xa1, como vai?' -> b'Hello, how are you?'
b'Esse \xc3\xa9 um teste.' -> b'This is a test.'
b'Estamos aprendendo Transformers.' -> b'We are learning Transformers.'


In [10]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

def load_data():
  dataset_name = "ted_hrlr_translate/pt_to_en"
  examples, metadata = tfds.load(dataset_name, as_supervised=True, with_info=True)
  return examples["train"], examples["validation"]

def tokenize_text(text, tokenizer):
  return tokenizer.encode(text.numpy())

def encode_map_fn(pt, en, tokenizer_pt, tokenizer_en):
  return tf.convert_to_tensor(tokenize_text(pt, tokenizer_pt), dtype=tf.int64), tf.convert_to_tensor(tokenize_text(en, tokenizer_en), dtype=tf.int64)

def prepare_data():
  train_data, val_data = load_data()
  tokenizer_pt = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
  (pt.numpy() for pt, en in train_data), target_vocab_size=213)
  tokenizer_en = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
  (en.numpy() for pt, en in train_data), target_vocab_size=213)
  train_data = train_data.map(lambda pt, en: encode_map_fn(pt, en, tokenizer_pt, tokenizer_en))
  val_data = val_data.map(lambda pt, en: encode_map_fn(pt, en, tokenizer_pt, tokenizer_en))
  return train_data, val_data, tokenizer_pt, tokenizer_en

In [11]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)
    depth = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(depth)
    if mask is not None:
        scaled_attention_logits += (mask * -1e9)
    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
    output = tf.matmul(attention_weights, v)
    return output

In [14]:
class Transformer(tf.keras.Model):
  def init(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, rate=0.1):
    super(Transformer, self).init()
    self.encoder = tf.keras.layers.Embedding(input_vocab_size, d_model)
    self.decoder = tf.keras.layers.Embedding(target_vocab_size, d_model)
    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inputs, training):
      inputs = tf.expand_dims(inputs, axis=-1)  # Adiciona uma dimensão extra
      enc_output = self.encoder(inputs)
      dec_output = self.decoder(inputs)
      final_output = self.final_layer(dec_output)
      return final_output

In [16]:
def train_model(model, dataset, epochs=10):
  optimizer = tf.keras.optimizers.Adam()
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  model.compile(optimizer=optimizer, loss=loss_object, metrics=["accuracy"])
  model.fit(dataset, epochs=epochs)

def evaluate_model(model, dataset):
  loss, acc = model.evaluate(dataset)
  print(f"Loss: {loss}, Accuracy: {acc}")

  if name == "main":
    train_data, val_data, tokenizer_pt, tokenizer_en = prepare_data()
    transformer = Transformer(num_layers=4, d_model=128, num_heads=8, dff=512, input_vocab_size=8000, target_vocab_size=8000)
    train_model(transformer, train_data)
    evaluate_model(transformer, val_data)

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

# ... (rest of your code, including load_data, encode_map_fn, prepare_data, scaled_dot_product_attention) ...

class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, rate=0.1):
        super(Transformer, self).__init__()
        self.encoder = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.decoder = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inputs, training=False):
        # inputs should be a tuple of (pt, en)
        enc_output = self.encoder(inputs[0])
        dec_output = self.decoder(inputs[1])

        # Apply the final layer to the decoder output
        final_output = self.final_layer(dec_output)
        return final_output


def train_model(model, dataset, epochs=10):
    optimizer = tf.keras.optimizers.Adam()
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(optimizer=optimizer, loss=loss_object, metrics=["accuracy"])
    history = model.fit(dataset, epochs=epochs)
    return history

def evaluate_model(model, dataset):
    loss, acc = model.evaluate(dataset)
    print(f"Loss: {loss}, Accuracy: {acc}")
    return loss, acc

def plot_results(history):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Loss')
    plt.xlabel('Épocas')
    plt.ylabel('Loss')
    plt.title('Evolução da Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Accuracy')
    plt.xlabel('Épocas')
    plt.ylabel('Accuracy')
    plt.title('Evolução da Accuracy')
    plt.legend()

    plt.show()

if __name__ == "__main__":
    train_data, val_data, tokenizer_pt, tokenizer_en = prepare_data()

    # Explicitly provide the input shape to the model
    transformer = Transformer(num_layers=4, d_model=128, num_heads=8, dff=512,
                              input_vocab_size=8000, target_vocab_size=8000)

    history = train_model(transformer, train_data)
    loss, acc = evaluate_model(transformer, val_data)
    plot_results(history)


